In [63]:
# paramter cell do not remove!!
# nb_parm='datalake|raw/pdf|Birddiversityanddistribution|pdf||300|150'
nb_parm='datalake|raw/text-csv|PFW_spp_translation_table_May2024|csv||300|150'
question = 'describe the characteristic of the brid from the follow context'
embed_model = "mxbai-embed-large" 
gen_model = "deepseek-r1:7b"
collection = "Bridcsvknowledge"

In [64]:
import sys
import os

sys.path.append("/home/jovyan/notebooks")
from Framework.module import Utility

In [65]:
## Do the task After this

In [66]:
bucket_name, path_to_file, file_name, file_type, dlm, chunk_size, overlap = nb_parm.split('|')
######PREPROCESSING###################
file_name = file_name + '.' + file_type
chunk_size = int(chunk_size)
overlap = int(overlap)
######################################
print("bucket_name:", bucket_name)
print("path_to_file:", path_to_file)
print("file_name:", file_name)
print("file_type:", file_type)
print("dlm:", dlm)
print("chunk_size:", chunk_size)
print("overlap:", overlap)
######################################
print("question:", question)
print("embed_model:", embed_model)
print("gen_model:", gen_model)
print("collection:", collection)


bucket_name: datalake
path_to_file: raw/text-csv
file_name: PFW_spp_translation_table_May2024.csv
file_type: csv
dlm: 
chunk_size: 300
overlap: 150
question: describe the characteristic of the brid from the follow context
embed_model: mxbai-embed-large
gen_model: deepseek-r1:7b
collection: csvbrid


In [67]:
client = Utility.registerClient()
if collection in client.collections.list_all():
    client.collections.delete(collection)
client.close()

In [68]:
import uuid
import weaviate
import pdfplumber
from weaviate.classes.config import Configure
from datetime import datetime
from weaviate.classes.config import Property, DataType
import io
import pandas as pd

OLLAMA_API = "http://host.docker.internal:11434"

def createCollection(client):
    # if present
    if collection in client.collections.list_all():
        return client.collections.get(collection)

    # if not
    return client.collections.create(
        name=collection,
        vectorizer_config=Configure.Vectorizer.text2vec_ollama(
            api_endpoint=OLLAMA_API,
            model=embed_model
        ),
        generative_config=Configure.Generative.ollama(
            api_endpoint=OLLAMA_API,
            model=gen_model
        ),
        properties=[
            Property(name="title", data_type=DataType.TEXT),
            Property(name="content", data_type=DataType.TEXT),
            Property(name="source_type", data_type=DataType.TEXT),
            Property(name="source_id", data_type=DataType.TEXT),
            Property(name="index", data_type=DataType.INT),
            Property(name="timestamp", data_type=DataType.DATE)
        ]
    )

def chunkText(text, chunk_size, overlap):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

def ingestFileToWeaviate(bucket_name, path_to_file, title, chunk_size, overlap):
    client = Utility.registerClient()

    kb = createCollection(client)
    
    # Get PDF file as bytes from MinIO
    file_bytes = Utility.readFileFromMinio(bucket_name, path_to_file)

    # Define as one document
    title = title
    source_id = str(uuid.uuid4())
    timestamp = datetime.utcnow().replace(microsecond=0).isoformat() + "Z"
    
    if file_type == 'pdf':
        with pdfplumber.open(io.BytesIO(file_bytes)) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    chunks = chunkText(text, chunk_size, overlap)
                    chunk_index = 0
                    for chunk in chunks:
                        kb.data.insert({
                            "title": title,
                            "content": chunk,
                            "source_type": file_type,
                            "source_id": source_id,
                            "index": chunk_index,
                            "timestamp": timestamp
                        })
                        chunk_index += 1
        print(f"Ingested {chunk_index} chunks from '{title}'")
                        
    elif file_type == 'txt' or file_type == 'csv':
        file_bytes = Utility.readFileFromMinio(bucket_name, path_to_file)
        df = pd.read_csv(io.StringIO(file_bytes.decode('utf-8')), on_bad_lines='skip')
        texts = df.astype(str).apply(lambda row: " | ".join(row), axis=1).tolist()
        text_index = 0
        for text in texts:
            kb.data.insert({
                "title": title,
                "content": text,
                "source_type": file_type,
                "source_id": source_id,
                "index": text_index,
                "timestamp": timestamp
            })
            text_index += 1
        print(f"Ingested {text_index} chunks from '{title}'")

    client.close()
    

 

In [ ]:
ingestFileToWeaviate(bucket_name, path_to_file+'/'+file_name, file_name, chunk_size, overlap)

In [76]:
client = Utility.registerClient()
for i in client.collections.list_all():
    print(client.collections.get(i))
    print("##################################################")
    print(i)
    print("\n\n break line here")
# client.collections.delete("Question")
# client.collections.delete("Csvbrid")
client.close()


In [ ]:
import weaviate
import json
from datetime import datetime
from weaviate.classes.query import MetadataQuery, Filter

def serialize(obj):
    """Helper function to handle serialization of datetime objects."""
    if isinstance(obj, datetime):
        return obj.isoformat()  # Convert datetime to string in ISO format
    raise TypeError("Type not serializable")

client = Utility.registerClient()

questions = client.collections.get(collection)

# Perform the text generation
# Each query must target one collection at a time
response = questions.query.bm25(
    query="Myioborus pictus",
    limit=1,
    query_properties=["content"],
    return_metadata=MetadataQuery(score=True),
)

for obj in response.objects:
    print(json.dumps(obj.properties, default=serialize, indent=2))
    print(json.dumps(obj.metadata.score, default=serialize, indent=2))


client.close()  # Free up resources


{
  "title": "PFW_spp_translation_table_May2024.csv",
  "source_id": "a397302f-df85-4ae7-9202-5ec3879f2f67",
  "index": 1056,
  "content": "paired | nan | 15 | Myioborus pictus | Painted Redstart | 2023 | 33810",
  "timestamp": "2025-04-09T06:19:40+00:00",
  "source_type": "csv"
}
5.711155891418457
